In [1]:
import os
import numpy as np
import torch
import torch.nn.functional as F

from detection_utils import random_label
from detection_evaluate import single_metric_fpr_tpr, combined_metric_fpr_tpr
from detection_baseline import targeted_vals, untargeted_vals

# from utils import transform
import torch.optim as optim

import torchvision
from torchvision import transforms
import torch.utils.data as data

from tqdm.notebook import tqdm
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import json

In [2]:
model = torchvision.models.resnet50(pretrained=True)

# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "/Users/albertwen/Downloads/mae_data/resnet50.pth"

# Setting the seed
# pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Fetching the device that will be used throughout this notebook
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)

pretrained_model = torchvision.models.resnet50(pretrained=True)
pretrained_model = pretrained_model.to(device)

if CHECKPOINT_PATH:
    if os.path.isfile(CHECKPOINT_PATH):
        checkpoint = torch.load(CHECKPOINT_PATH) 
        # model.load_state_dict(checkpoint['state_dict'])  #['state_dict']
        pretrained_model.load_state_dict(checkpoint)  #['state_dict']
        print("=> load checkpoint found at {}".format(CHECKPOINT_PATH))
        # print("=> loaded checkpoint '{}' (epoch {})"
        #       .format(args.resume, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(CHECKPOINT_PATH))
# No gradients needed for the network
pretrained_model.eval()
for p in pretrained_model.parameters():
    p.requires_grad = False

Using device cpu
=> load checkpoint found at /Users/albertwen/Downloads/mae_data/resnet50.pth


In [3]:
fpr = 0.1 # can be [0.1, 0.2]
untargeted_step_threshold = 1000
criterions = {0.1: (0.190, 35, untargeted_step_threshold), 0.2: (1.77, 22, untargeted_step_threshold)} # pre-defined in paper's code
dataset = "imagenet"
attacks = ["fgsm_4", "fgsm_8", "pgd_8"]
real_d = "/Users/albertwen/Downloads/mae_data/ImageNet-Data/val"
adv_d = "/Users/albertwen/Downloads/mae_data/ImageNet-Data/attack/val/"

noise_radius = 0.1
targeted_lr = 0.005
targeted_radius = 0.03
untargeted_radius = 0.03
untargeted_lr = 0.1

det_opt = "untargeted" # can be ["targeted", "untargeted"]

In [4]:
single_metric_fpr_tpr(fpr, 
                      criterions, 
                      pretrained_model, 
                      dataset, 
                      attacks, 
                      real_d, 
                      adv_d, 
                      noise_radius, 
                      targeted_lr, 
                      targeted_radius, 
                      untargeted_lr, 
                      untargeted_radius, 
                      opt=det_opt)